In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU

import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.nn import NNConv, Set2Set
from torch_geometric.data import DataLoader
from torch_geometric.utils import remove_self_loops

In [3]:
class MyTransform(object):
    def __call__(self, data):
        # Specify target.
        data.y = data.y[:, target]
        return data


class Complete(object):
    def __call__(self, data):
        device = data.edge_index.device

        row = torch.arange(data.num_nodes, dtype=torch.long, device=device)
        col = torch.arange(data.num_nodes, dtype=torch.long, device=device)

        row = row.view(-1, 1).repeat(1, data.num_nodes).view(-1)
        col = col.repeat(data.num_nodes)
        edge_index = torch.stack([row, col], dim=0)

        edge_attr = None
        if data.edge_attr is not None:
            idx = data.edge_index[0] * data.num_nodes + data.edge_index[1]
            size = list(data.edge_attr.size())
            size[0] = data.num_nodes * data.num_nodes
            edge_attr = data.edge_attr.new_zeros(size)
            edge_attr[idx] = data.edge_attr

        edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
        data.edge_attr = edge_attr
        data.edge_index = edge_index

        return data


path = osp.join(osp.dirname(osp.realpath('__file__')), '..', 'data', 'QM9')
transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])
dataset = QM9(path, transform=transform).shuffle()



In [6]:
import numpy as np

In [16]:
1125*np.log(1+25600/700)

4079.5243685127575

In [24]:
700*(np.exp2(4079.5243685127575/1125)-1)

4672740504900.22

In [17]:
1125*np.log(1+0/700)

0.0

In [28]:
for m in np.linspace(0, 4079.5243685127575, num=9):
    h = 700*(np.exp(m/1125)-1)
    fft_bin = np.floor((4096+1)*h/51200)
    print(m, '==============>',h, '==============>',fft_bin)

0.0 ==============> 0.0 ==============> 0.0
509.9405460640947 ==============> 401.425821089579 ==============> 32.0
1019.8810921281894 ==============> 1033.0554848040758 ==============> 82.0
1529.8216381922841 ==============> 2026.9029433487538 ==============> 162.0
2039.7621842563788 ==============> 3590.687590584987 ==============> 287.0
2549.7027303204736 ==============> 6051.248717855625 ==============> 484.0
3059.6432763845683 ==============> 9922.856660634421 ==============> 794.0
3569.583822448663 ==============> 16014.698028223103 ==============> 1281.0
4079.5243685127575 ==============> 25600.0 ==============> 2048.0


In [4]:
# Normalize targets to mean = 0 and std = 1.
mean = dataset.data.y[:, target].mean().item()
std = dataset.data.y[:, target].std().item()
dataset.data.y[:, target] = (dataset.data.y[:, target] - mean) / std

# Split datasets.
test_dataset = dataset[:100]
val_dataset = dataset[100:200]
train_dataset = dataset[200:300]
test_loader = DataLoader(test_dataset, batch_size=64)
val_loader = DataLoader(val_dataset, batch_size=64)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

NameError: name 'target' is not defined

In [ ]:
train_dataset

In [88]:
for i in range(13):
    print(i, train_dataset.data.x[:,i].std(), train_dataset.data.x[:,i].mean())

0 tensor(0.4999) tensor(0.5159)
1 tensor(0.4771) tensor(0.3499)
2 tensor(0.2239) tensor(0.0529)
3 tensor(0.2700) tensor(0.0791)
4 tensor(0.0472) tensor(0.0022)
5 tensor(2.7774) tensor(3.6384)
6 tensor(0.2818) tensor(0.0869)
7 tensor(0.2052) tensor(0.0440)
8 tensor(0.2316) tensor(0.0568)
9 tensor(0.1547) tensor(0.0245)
10 tensor(0.3481) tensor(0.1409)
11 tensor(0.4661) tensor(0.3187)
12 tensor(0.8643) tensor(0.5159)


In [19]:
for data in train_loader:
    break

In [42]:
target = 0
dim = 32

In [69]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin0 = torch.nn.Linear(dataset.num_features, dim)

        nn = Sequential(Linear(5, 128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean', root_weight=False)
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
#         print(out.shape)
        h = out.unsqueeze(0)
#         print(h.shape)

        for i in range(3):
#             print('edge_index', data.edge_index.shape, 'edge_attr':data.edge_attr.shape)
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
#             print(m.shape)
            out, h = self.gru(m.unsqueeze(0), h)
#             print(out.shape, h.shape)
            out = out.squeeze(0)
#             print(out.shape)

        out = self.set2set(out, data.batch)
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
        out = out.view(-1)
#         print(out.shape)
        return out


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, min_lr=0.00001)


def train(epoch):
    model.train()
    loss_all = 0

    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
#         print(i, data.y.shape)
        loss = F.mse_loss(model(data), data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_loader.dataset)


def test(loader):
    model.eval()
    error = 0

    for data in loader:
        data = data.to(device)
        error += (model(data) * std - data.y * std).abs().sum().item()  # MAE
    return error / len(loader.dataset)




In [70]:
best_val_error = None
for epoch in range(1, 2):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = train(epoch)
    val_error = test(val_loader)
    scheduler.step(val_error)

    if best_val_error is None or val_error <= best_val_error:
        test_error = test(test_loader)
        best_val_error = val_error

    print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Validation MAE: {:.7f}, Test MAE: {:.7f}'.format(epoch, lr, loss, val_error, test_error))

Epoch: 001, LR: 0.001000, Loss: 0.7713466, Validation MAE: 1.1874266, Test MAE: 1.0079571
